In [38]:
import sqlite3
import numpy as np
import pandas as pd
import tensorflow as tf
import re

In [39]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("database.sqlite")
reviews = pd.read_sql_query("SELECT * from reviews", con)

podcasts = pd.read_sql_query("SELECT * from podcasts", con)
podcasts.rename(columns={'title': 'podcast_title'}, inplace = True)

categories = pd.read_sql_query("SELECT * from categories", con)

In [40]:
# Deal with multiple categories problem
def get_main_cat(line):
    line = line.replace(line, re.findall(r'\b([a-zA-Z]+)\b', line)[0])
    
    return line

In [41]:
categories['category'] = categories['category'].apply(lambda x: get_main_cat(x))

In [42]:
categories['category'].unique()

array(['arts', 'music', 'education', 'society', 'leisure', 'technology',
       'fiction', 'true', 'health', 'history', 'comedy', 'tv', 'kids',
       'religion', 'business', 'news', 'spirituality', 'science',
       'christianity', 'government', 'sports', 'hinduism', 'judaism',
       'islam', 'buddhism'], dtype=object)

In [43]:
def replace_cat(line):
    line = re.sub(r'\b(true)\b', 'crime', line)
    line = re.sub(r'\b(christianity|hinduism|judaism|islam|buddhism|spirituality)\b', 'religion', line)
    return line

In [44]:
categories['category'] = categories['category'].apply(lambda x: replace_cat(x))
categories.drop_duplicates(subset='podcast_id', keep="first", inplace = True)
categories.head()

,podcast_id,category
0,c61aa81c9b929a66f0c1db6cbe5d8548,arts
3,ad4f2bf69c72b8db75978423c25f379e,arts
7,f2970432aad718a000ebac10e48ae6b0,arts
10,f90b974d49b86b2d4d1769519fb478c0,arts
14,d9122defb9898145688449f610c8943e,arts


In [45]:
df = pd.merge(reviews, podcasts, on=['podcast_id'], how = 'inner')
df = pd.merge(df, categories, on=['podcast_id'], how = 'inner')
df.shape

(1979519, 11)

In [ ]:
# sample

In [73]:
print(df[df['rating'] == 1].shape)
print(df[df['rating'] == 2].shape)
print(df[df['rating'] == 3].shape)
print(df[df['rating'] == 4].shape)
print(df[df['rating'] == 5].shape)

(110013, 11)
(43213, 11)
(49215, 11)
(59665, 11)
(1717413, 11)


In [62]:
df_5 = df[df['rating'] == 5].sample(200000)

In [64]:
df_1 = df[df['rating'] == 1]
df_2 = df[df['rating'] == 2]
df_3 = df[df['rating'] == 3]
df_4 = df[df['rating'] == 4]

In [65]:
df_bf_s = pd.concat([df_1, df_2, df_3, df_4, df_5])
df_bf_s.shape

(462106, 11)

In [82]:
df_s = df_bf_s.sample(50000)
df_s.reset_index(drop = True, inplace = True)
df_s.shape

(50000, 11)

In [83]:
print(df_s[df_s['rating'] == 1].shape)
print(df_s[df_s['rating'] == 2].shape)
print(df_s[df_s['rating'] == 3].shape)
print(df_s[df_s['rating'] == 4].shape)
print(df_s[df_s['rating'] == 5].shape)

(11830, 11)
(4622, 11)
(5299, 11)
(6512, 11)
(21737, 11)


In [84]:
df_s.to_csv('podcast_sample.csv')

In [85]:
test = pd.read_csv('podcast_sample.csv', lineterminator='\n', index_col = 0)
test.head()

,podcast_id,title,content,rating,author_id,created_at,itunes_id,slug,itunes_url,podcast_title,category
0,b313ef8ef0d5b64290d3036ff1bbf2d2,감성 라디오 음악도시,미국 서부에 있는 유학생이에요. 성시경씨 제대 후 라디오 복귀만 기다려오다가 6 월...,5,664CCA7142E9AE8,2011-09-14T13:25:46-07:00,442838670,fm-%EC%9D%8C%EC%95%85%EB%8F%84%EC%8B%9C-%EC%A2...,https://podcasts.apple.com/us/podcast/fm-%EC%9...,FM 음악도시(종영),music
1,abfb842993be20d21bfae7103addc5e9,They’ve really cut back on the content this se...,Last season there was a new pod every 3-4 days...,1,AD790CE113DCBC1,2018-04-11T13:46:47-07:00,1015394113,the-good-phight-for-philadelphia-phillies-fans,https://podcasts.apple.com/us/podcast/the-good...,The Good Phight: for Philadelphia Phillies fans,sports
2,ebdf879a424547d01862a9bbba18a0f3,Good info. source...,Bob brings a lot of knowledge to any firearm d...,4,E223A4B2642C970,2010-01-19T08:11:43-07:00,333180229,handgun-world-podcast,https://podcasts.apple.com/us/podcast/handgun-...,Handgun World Podcast,news
3,ab2fdb7db023b223d870487165d11ff3,Mixed,They have lost much of thier credibility by de...,3,E1E7DBE750D119E,2021-01-28T12:21:49-07:00,971901464,wsj-opinion-potomac-watch,https://podcasts.apple.com/us/podcast/wsj-opin...,WSJ Opinion: Potomac Watch,news
4,ca601bd1524322d0527b16adf2738ff3,Try it now!,Even better than I expected. I was interested ...,5,D7CA4858AFA2CFC,2017-08-24T10:55:20-07:00,1257821731,conversations-with-people-who-hate-me,https://podcasts.apple.com/us/podcast/conversa...,Conversations with People Who Hate Me,society


In [86]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   podcast_id     50000 non-null  object
 1   title          50000 non-null  object
 2   content        50000 non-null  object
 3   rating         50000 non-null  int64 
 4   author_id      50000 non-null  object
 5   created_at     50000 non-null  object
 6   itunes_id      50000 non-null  int64 
 7   slug           50000 non-null  object
 8   itunes_url     50000 non-null  object
 9   podcast_title  49999 non-null  object
 10  category       50000 non-null  object
dtypes: int64(2), object(9)
memory usage: 4.6+ MB
